# A simple recommender example using Python


https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/

A very friendly explanation on Youtube:
https://www.youtube.com/watch?v=ZspR5PZemcs

## 1. Data

The quantity and quality of the data determines the quality of predictions. 
Gather enough data, refine (extract) relevant subset,...

### 2.1 Content-based filtering

Assume that if a user like A, and B is similar to A, then the user will probably like B, too.
Consider a *profile vector*, which contains behavior of a Netflix user. This vector can be compared to *item vectors* of videos containing informations such as genre, cast, directors, etc. 


#### Measures
The similarity can be calculated as the cosine between vectors as
#### Cosine similarity

\begin{equation*}
 sim(A,B) = cos(\theta) = \frac{A \dot B}{\left \| A \right \|\left \| B \right \|}
\end{equation*}

#### Euclidean distance

#### Pearson's Correlation

\begin{equation*}
sim(u,v)=\frac { \sum { ({ r }_{ ui }-{ \bar { r }  }_{ u })({ r }_{ vi }-\bar { { r }_{ v } } ) }  }{ \sqrt { { \sum { ({ r }_{ ui }-{ \bar { r }  }_{ u }) }  }^{ 2 } } \sqrt { { \sum { ({ r }_{ vi }-\bar { { r }_{ v } } ) }  }^{ 2 } }  } 
\end{equation*}



#### Draw back
This method can only be applied to the data set of the *same* kind as the user has rated.

### 2.2 More general, collaborative filtering

#### Between users
If user A and B both like a,b, and B also like c, then A may like c, too. 

$P_{u,i}$, the prediction of an item i for a user u can be calculated as

\begin{equation*}
{ P }_{ u,i }=\frac { \sum _{ v }^{  }{ \left( { r }_{ v,i }*{ s }_{ u,v } \right)  }  }{ \sum _{ v }^{  }{ { s }_{ u,v } }  } 
\end{equation*}

, where v denotes other user, and S represents the similarity between users.

Calculating similiarty between all pairs of users is time consuming. Thus a subset of users might be used in practice.

The same can be done **between items.**

# Case study

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
u_cols = ["user_id", "age", "sex", 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding="latin-1")

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding="latin-1")
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

i_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
          "unknown", 'Action', 'Adventure', 'Animation', 'Children\'s',
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
          'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

print(ratings_train.shape, ratings_test.shape)

(90570, 4) (9430, 4)


## Collaborative filtering model
 both user similarity and item similarity.

In [3]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

In [4]:
# -1 because pandas index starts from 1.

data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [5]:
from sklearn.metrics.pairwise import pairwise_distances
"""
    Distance metrics ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']
"""

# user-user similarity, item-item similarity
user_similarity = pairwise_distances(data_matrix, metric="cosine")
item_similarity = pairwise_distances(data_matrix.T, metric="cosine")

In [6]:
def predict(ratings, similarity, type="user"):
    "Predicted rating based on "
    if type=='user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + \
                similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        
    return pred

In [7]:
u_pred = predict(data_matrix, user_similarity, type="user")
i_pred = predict(data_matrix, item_similarity, type="item")

In [9]:
import turicreate

In [10]:
train_data = turicreate.SFrame(ratings_train)
test_data = turicreate.SFrame(ratings_test)

In [13]:
# Most popular one
popularity_model = turicreate.recommender.popularity_recommender.create(train_data, user_id='user_id',
                                                                       item_id="movie_id", target='rating')

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.041771s

90570 observations to process; with 1680 unique items.

In [16]:
popularity_recomm = popularity_model.recommend(users=[1,2,3,4,5], k=5)
# show top 5 recommendations.
popularity_recomm.print_rows()
# All users get the same recommendations

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1500   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    2    |   1500   |  5.0  |  1   |
|    2    |   1201   |  5.0  |  2   |
|    2    |   1189   |  5.0  |  3   |
|    2    |   1122   |  5.0  |  4   |
|    2    |   814    |  5.0  |  5   |
+---------+----------+-------+------+
[25 rows x 4 columns]



### Collaborative filtering model

In [18]:
# By item similarity
item_sim_model = turicreate.item_similarity_recommender.create(train_data, user_id="user_id", 
                                                              item_id='movie_id', target='rating',
                                                              similarity_type='cosine')

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.048134s

...

| 73.136ms                            | 100              | 1680            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.078969s

In [21]:
item_sim_recomm = item_sim_model.recommend(users=[1,2,3,4,5], k=5)
item_sim_recomm.print_rows()

+---------+----------+--------------------+------+
| user_id | movie_id |       score        | rank |
+---------+----------+--------------------+------+
|    1    |   423    | 0.980611449434557  |  1   |
|    1    |   202    | 0.9542551061124293 |  2   |
|    1    |   655    | 0.7947521701113869 |  3   |
|    1    |   403    | 0.765623665037956  |  4   |
|    1    |   568    | 0.7620385562190573 |  5   |
|    2    |    50    | 1.1256258487701416 |  1   |
|    2    |   181    | 1.0382135510444641 |  2   |
|    2    |    7     | 0.9527609990193293 |  3   |
|    2    |   121    | 0.9145556126649563 |  4   |
|    2    |    9     | 0.8531226699168866 |  5   |
+---------+----------+--------------------+------+
[25 rows x 4 columns]



Different users get different recommendations based on each user's preference.

#### By user similarity

To compare movie ratings ny different users, we want ratings of all movies by every user.
Of course, **not** every user has rated every movies. We will guess missing ratings of each user based on a set of other features of the user (called **latent features**.)
To extract the most useful latent features from the existing features, we will use matric factorization. 

### Matrix factorization

Matrix factorization is a general term. PCA is an example of matrix factorization. 
https://sites.google.com/site/igorcarron2/matrixfactorizations contains tons of information about matrix factorization. 


We decompose the complete rating matrix R as,  

\begin{equation*}
 R = P \Sigma Q^{\intercal}
\end{equation*}

Here, R is the complete user $\times$ movies rating matrix (M $\times$ N),  
P is M $\times$ K user-feature affinity matrix which represents the association between users and (latent) features,  
Q is N $\times$ K latent feature - movie relevance matrix,  
$\Sigma$ is K$\times$K feature weight matrix which represents the essential weights of features. (Not summation!)

So, how do we determine R and Q?  
In this example, we use **gradient descent** algorithm.

We want to minimize the sum of the errors between the actual rating $r_{ui}$ and the predicted rating $\check{r}_{ui}$ as

\begin{equation*}
{e_{ui}}^2 = (r_{ui} - \check{r}_{ui})^2 = (r_{ui} - \sum _{k=1 }^{K }{ p_{uk} \sigma_k q_{ki} })^2
\end{equation*}  


In the gradient descent algorithm, p and q are updated so that the gradient of error is minimized. 

\begin{align*}
\frac { \partial  }{ \partial { p }_{ uk } } ({ { e }_{ ui } }^{ 2 }) = -2({ r }_{ ui }-{ \check { r }  }_{ ui }){ q }_{ ki }=-2{ e }_{ ui }{ q }_{ ki } \\
\frac { \partial  }{ \partial { q }_{ ki } } ({ { e }_{ ui } }^{ 2 }) = -2({ r }_{ ui }-{ \check { r }  }_{ ui }){ p }_{ uk }=-2{ e }_{ ui }{ p }_{ uk }
\end{align*}

Now, let the system evolve according to the *update rule*, or PDEs!

In [67]:
class MF():
    """
    R = rating matrix
    K = Numbre of latent features
    alpha = Learning rate (delta one step = alpha * gradient)
    beta = Regularization parameter for bias
    
    P = prediction
    """
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    # Initialize
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initializing the bias terms ???
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        #self.b = np.mean(self.R[np.where(self.R != 0)])
        self.b = np.mean(self.R[self.R != 0])
        
        self.samples = [
            (i,j, self.R[i,j]) 
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i,j] >0
        ]
        
        # Stochastic gradient descent for given number of iterations
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i,mse))
            if i+1 % 20 == 0:
                print("Iteration: %d ; error = %.4f".format(i+1, mse))
                
        return training_process
    
    def mse(self):
        """
        Mean squared error
        """
        i_missing = self.R.nonzero() # Return the indices of the elements that are non-zero.
        predicted = self.full_matrix()
        return np.sqrt(np.sum(np.square(self.R[i_missing] - predicted[i_missing])))
    
    
    def sgd(self):
        """
            stochastic gradient descent 
        """
        for i,j,r in self.samples:
            prediction = self.get_rating(i,j)
            e = r - prediction
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.alpha * self.b_i[j])
            self.P[i,:] += self.alpha * (e * self.Q[j,:] - self.beta * self.P[i,:])
            self.Q[j,:] += self.alpha * (e * self.P[i,:] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        return self.b + self.b_u[i] + self.b_i[j] + \
               np.dot(self.P[i,:], self.Q[j,:].T)
    
    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + \
               self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [71]:
# convert user ratings to matrix form.
# pd.pivot
#   Return reshaped DataFrame organized by given index / column values.
R = np.array(ratings.pivot(index = 'user_id', 
                           columns = 'movie_id', 
                           values ='rating').fillna(0))
print(R.shape)

(943, 1682)


In [72]:
mf = MF(R, K=20, alpha=0.001, beta=0.01, iterations=100)

In [73]:
mf.train()

[(0, 337.59117477665035),
 (1, 327.02008959794387),
 (2, 320.24510881421156),
 (3, 315.54492665646654),
 (4, 312.08358682395675),
 (5, 309.42212419737),
 (6, 307.3030738108998),
 (7, 305.5683580542252),
 (8, 304.1174025784298),
 (9, 302.88265944451115),
 (10, 301.8134740002398),
 (11, 300.87640113094875),
 (12, 300.046107018827),
 (13, 299.3038402990994),
 (14, 298.6349423254205),
 (15, 298.02785798359264),
 (16, 297.4740190745409),
 (17, 296.96603926810565),
 (18, 296.4983368970952),
 (19, 296.06504845826515),
 (20, 295.6623642321388),
 (21, 295.28725924804337),
 (22, 294.9355023340405),
 (23, 294.6055551163985),
 (24, 294.29514775581606),
 (25, 294.0014235816799),
 (26, 293.72355944036326),
 (27, 293.4596487152364),
 (28, 293.2086844883392),
 (29, 292.9689578424467),
 (30, 292.73937760933774),
 (31, 292.51898357860375),
 (32, 292.3075311762075),
 (33, 292.10380164233356),
 (34, 291.9062459331664),
 (35, 291.71488430253834),
 (36, 291.52967813881537),
 (37, 291.3488578410917),
 (38, 2

In [95]:
print("Predictied ratings")
print(mf.full_matrix()[mf.R.nonzero()])

Predictied ratings
[4.00607722 3.3241638  3.09252022 ... 2.88405447 2.45350727 3.18294452]


To do, 

What is the bias??

## Evaluation

### metrics used

1. Recall
\begin{equation*}
 R =\frac{good\, prediction}{good\, prediction + good\, given}
\end{equation*}

Better prediction with smaller data means higher recall.
But, if the engine recommended everything, the Recall is high.

2. Prediction
\begin{equation*}
 P =\frac{good\, prediction}{total\, prediction}
\end{equation*}


So, recall ~ coverage, prediction ~ success rate.

3. RMS error
\begin{equation*}
RMSE=\sqrt { \frac { \sum _{ i=1 }^{ N }{ (predicted_{ i }-Actual_{ i }) }  }{ N }  } 
\end{equation*}

#### ranking metrics (more weights on more favored recommendation)
4. Mean Reciprocal Rank
\begin{equation*}
MRR=\frac { 1 }{ N } \sum _{ i=1 }^{ N }{ \frac { 1 }{ r({ Q }_{ i }) }  }
\end{equation*}
, where r is the rank, Q maybe a *non-ranked* metric such as R or P.


5. MAP at K (Mean Average Precision at cutoff K)
\begin{equation*}
{ MAP }_{ i }=\frac { 1 }{ \left| { R }_{ i } \right|  } \sum _{ k=1 }^{ R_{ i } }{ P({ R }_{ i }\left[ k \right] ) } 
\end{equation*}